In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [9]:
original = pd.read_csv('airlinedelaycauses_DelayedFlights.csv')

In [10]:
original.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = original.copy()

In [12]:
data = data.rename(columns={'Unnamed: 0': 'ID'})

In [19]:
data['DayofMonth'].max()

31

In [22]:
# merge Year, Month, DayofMonth and  DayOfWeek into a single Date column
data['Date'] = pd.to_datetime(dict(
    year=data['Year'],
    month=data['Month'],
    day=data['DayofMonth']
))


In [24]:
data.drop(columns=['Year', 'Month', 'DayofMonth'], inplace=True)

In [28]:
data.rename(columns={'CRSDepTime': 'ScheduledDepTime', 'DepTime' : 'ActualDepTime', 'CRSArrTime': 'ScheduledArrTime', 'ArrTime': 'ActualArrTime', 'CRSElapsedTime' : 'ScheduledElapsedTime'}, inplace=True)

In [45]:
data['CancellationReason'] = data['CancellationCode'].map({
    'N': 'Not Cancelled',
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'NAS'
})
data.drop(columns=['CancellationCode'], inplace=True)

In [50]:
data.columns

Index(['ID', 'DayOfWeek', 'ActualDepTime', 'ScheduledDepTime', 'ActualArrTime',
       'ScheduledArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'ScheduledElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay', 'Date', 'CancellationReason'],
      dtype='object')

In [55]:
data['Diverted']

0          0
1          0
2          0
3          0
4          0
          ..
1936753    0
1936754    0
1936755    0
1936756    0
1936757    0
Name: Diverted, Length: 1936758, dtype: int64

In [61]:
data['ScheduledDepTime']

0          1955
1           735
2           620
3          1755
4          1915
           ... 
1936753    1220
1936754     600
1936755     847
1936756    1240
1936757    1103
Name: ScheduledDepTime, Length: 1936758, dtype: int64

In [ ]:
for col in ['ActualDepTime', 'ScheduledDepTime', 'ActualArrTime', 'ScheduledArrTime']:
    times = (
        data[col]
        .fillna(0)
        .astype(int)
        .astype(str)
        .str.zfill(4)
        .replace('2400', '0000')
    )
    data[col] = pd.to_datetime(times, format='%H%M', errors='coerce').dt.time


In [69]:
data['Date'] = pd.to_datetime(data['Date'])

In [72]:
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest', 'CancellationReason', 'DayOfWeek']
data[categorical_cols] = data[categorical_cols].astype('category')

In [76]:
data['Cancelled'] = data['Cancelled'].astype(bool)
data['Diverted'] = data['Diverted'].astype(bool)

In [ ]:
delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
data[delay_cols] = data[delay_cols].fillna(0)

In [109]:
data.isnull().sum()

ID                         0
DayOfWeek                  0
ActualDepTime              0
ScheduledDepTime           0
ActualArrTime              0
ScheduledArrTime           0
UniqueCarrier              0
FlightNum                  0
TailNum                    5
ActualElapsedTime       8387
ScheduledElapsedTime     198
AirTime                 8387
ArrDelay                8387
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                  7110
TaxiOut                  455
Cancelled                  0
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
Date                       0
CancellationReason         0
dtype: int64

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 28 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   DayOfWeek             category      
 2   ActualDepTime         object        
 3   ScheduledDepTime      object        
 4   ActualArrTime         object        
 5   ScheduledArrTime      object        
 6   UniqueCarrier         category      
 7   FlightNum             int64         
 8   TailNum               object        
 9   ActualElapsedTime     float64       
 10  ScheduledElapsedTime  float64       
 11  AirTime               float64       
 12  ArrDelay              float64       
 13  DepDelay              float64       
 14  Origin                category      
 15  Dest                  category      
 16  Distance              int64         
 17  TaxiIn                float64       
 18  TaxiOut               float64       
 19  

In [112]:
data.to_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv', index=False)

In [3]:
data_cleaned = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Visualizations

In [32]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Load and preprocess data
data = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Handle ScheduledDepTime
if data['ScheduledDepTime'].dtype == 'object':
    try:
        data['Hour'] = pd.to_datetime(data['ScheduledDepTime'], format='%H:%M:%S', errors='coerce').dt.hour
    except:
        data['Hour'] = data['ScheduledDepTime'].astype(str).str.split(':').str[0]
        data['Hour'] = pd.to_numeric(data['Hour'], errors='coerce').fillna(0).astype(int)
else:
    data['ScheduledDepTime'] = pd.to_numeric(data['ScheduledDepTime'], errors='coerce')
    data['Hour'] = (data['ScheduledDepTime'] // 100).fillna(0).astype(int)

data['Hour'] = data['Hour'].clip(0, 23)
data['DayOfWeek'] = data['DayOfWeek'].astype(int)
data['Date'] = pd.to_datetime(data['Date'])

day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 
             5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
data['DayName'] = data['DayOfWeek'].map(day_names)

# Initialize Dash app
app = Dash(__name__)

# Enhanced color scheme
colors = {
    'background': '#f5f7fa',
    'card': '#ffffff',
    'text': '#2c3e50',
    'primary': '#3498db',
    'secondary': '#e74c3c',
    'success': '#2ecc71',
    'warning': '#f39c12',
    'info': '#16a085',
    'border': '#e1e8ed'
}

# Layout
app.layout = html.Div(style={
    'backgroundColor': colors['background'], 
    'minHeight': '100vh',
    'padding': '0',
    'margin': '0',
    'fontFamily': '-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif'
}, children=[
    
    # Header
    html.Div(style={
        'backgroundColor': colors['primary'],
        'padding': '30px 40px',
        'boxShadow': '0 2px 8px rgba(0,0,0,0.1)',
        'marginBottom': '30px'
    }, children=[
        html.Div(style={'maxWidth': '1400px', 'margin': '0 auto'}, children=[
            html.H1("✈️ Flight Delay Analytics", 
                    style={'color': 'white', 'margin': '0', 'fontSize': '36px', 'fontWeight': '600'}),
            html.P("Explore patterns and insights in airline delays", 
                   style={'color': 'rgba(255,255,255,0.9)', 'margin': '10px 0 0 0', 'fontSize': '16px'})
        ])
    ]),
    
    # Main container
    html.Div(style={'maxWidth': '1400px', 'margin': '0 auto', 'padding': '0 20px 40px 20px'}, children=[
        
        # Control Panel - Collapsible
        html.Div(style={
            'backgroundColor': colors['card'],
            'padding': '25px',
            'borderRadius': '12px',
            'marginBottom': '25px',
            'boxShadow': '0 1px 3px rgba(0,0,0,0.08)',
            'border': f'1px solid {colors["border"]}'
        }, children=[
            html.Div([
                html.H3("🎛️ Dashboard Controls", 
                       style={'color': colors['text'], 'margin': '0 0 20px 0', 'fontSize': '20px', 'fontWeight': '600'}),
                
                # Quick Actions
                html.Div([
                    html.Button("Select All Airlines", id='select-all-btn',
                               style={'backgroundColor': colors['success'], 'color': 'white', 'border': 'none',
                                      'padding': '8px 16px', 'borderRadius': '6px', 'cursor': 'pointer',
                                      'marginRight': '10px', 'fontSize': '14px', 'fontWeight': '500'}),
                    html.Button("Clear All Airlines", id='clear-all-btn',
                               style={'backgroundColor': colors['secondary'], 'color': 'white', 'border': 'none',
                                      'padding': '8px 16px', 'borderRadius': '6px', 'cursor': 'pointer',
                                      'marginRight': '10px', 'fontSize': '14px', 'fontWeight': '500'}),
                    html.Button("Reset to Defaults", id='reset-btn',
                               style={'backgroundColor': colors['info'], 'color': 'white', 'border': 'none',
                                      'padding': '8px 16px', 'borderRadius': '6px', 'cursor': 'pointer',
                                      'fontSize': '14px', 'fontWeight': '500'})
                ], style={'marginBottom': '20px'}),
                
                # Filter row
                html.Div([
                    # Delay Type
                    html.Div([
                        html.Label("📊 Delay Type", 
                                  style={'fontWeight': '600', 'marginBottom': '8px', 'display': 'block', 
                                        'color': colors['text'], 'fontSize': '14px'}),
                        dcc.Dropdown(
                            id='delay-type-dropdown',
                            options=[
                                {'label': '🛬 Arrival Delay', 'value': 'ArrDelay'},
                                {'label': '🛫 Departure Delay', 'value': 'DepDelay'}
                            ],
                            value='ArrDelay',
                            clearable=False,
                            style={'width': '100%', 'fontSize': '14px'}
                        )
                    ], style={'width': '13%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
                    
                    # Airlines
                    html.Div([
                        html.Label("🏢 Select Airlines", 
                                  style={'fontWeight': '600', 'marginBottom': '8px', 'display': 'block', 
                                        'color': colors['text'], 'fontSize': '14px'}),
                        dcc.Dropdown(
                            id='airline-filter',
                            options=[{'label': f'{carrier}', 'value': carrier} 
                                    for carrier in sorted(data['UniqueCarrier'].unique())],
                            value=sorted(data['UniqueCarrier'].unique()),
                            multi=True,
                            placeholder="Select airlines...",
                            style={'width': '100%', 'fontSize': '14px'}
                        )
                    ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
                    
                   # Date Range
                    html.Div([
                        html.Label("📅 Date Range", 
                                  style={'fontWeight': '600', 'marginBottom': '8px', 'display': 'block', 
                                        'color': colors['text'], 'fontSize': '14px'}),
                        dcc.DatePickerRange(
                            id='date-range',
                            start_date=data['Date'].min(),
                            end_date=data['Date'].max(),
                            display_format='MMM DD, YYYY',
                            style={'width': '100%', 'fontSize': '12px'},
                            className='custom-date-picker'
                        )
                    ], style={'width': '23%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
                    
                    # Delay Threshold with value display
                    html.Div([
                        html.Div([
                            html.Label("⏱️ Minimum Delay Threshold", 
                                      style={'fontWeight': '600', 'display': 'inline-block', 
                                            'color': colors['text'], 'fontSize': '14px'}),
                            html.Span(id='threshold-value', 
                                     style={'float': 'right', 'fontWeight': '600', 
                                           'color': colors['primary'], 'fontSize': '14px'})
                        ], style={'marginBottom': '8px'}),
                        dcc.Slider(
                            id='delay-threshold',
                            min=0,
                            max=120,
                            step=5,
                            value=120,
                            marks={0: '0m', 30: '30m', 60: '1h', 90: '90m', 120: '2h'},
                            tooltip={"placement": "bottom", "always_visible": False}
                        )
                    ], style={'width': '23%', 'display': 'inline-block', 'verticalAlign': 'top'})
                ], style={'marginTop': '10px'})
            ]),
            
            # Info message
            html.Div([
                html.Div("💡", style={'display': 'inline-block', 'marginRight': '10px', 'fontSize': '20px'}),
                html.Span("Tip: Hover over charts for detailed information. Click on legend items to show/hide data.", 
                         style={'color': '#7f8c8d', 'fontSize': '13px'})
            ], style={'marginTop': '20px', 'padding': '12px', 'backgroundColor': '#ecf9ff', 
                     'borderRadius': '6px', 'border': '1px solid #b3e5fc'})
        ]),
        
        # Loading indicator
        dcc.Loading(
            id="loading",
            type="circle",
            color=colors['primary'],
            children=[
                # Summary Statistics
                html.Div(id='summary-stats', style={'marginBottom': '25px'}),
                
                # Tabs for organized content
                dcc.Tabs(id='main-tabs', value='overview', 
                        style={'marginBottom': '20px'},
                        children=[
                    dcc.Tab(label='📈 Overview', value='overview', 
                           style={'padding': '12px', 'fontWeight': '600'},
                           selected_style={'padding': '12px', 'fontWeight': '600', 
                                         'borderTop': f'3px solid {colors["primary"]}'}),
                    dcc.Tab(label='🏢 Airlines & Routes', value='airlines',
                           style={'padding': '12px', 'fontWeight': '600'},
                           selected_style={'padding': '12px', 'fontWeight': '600', 
                                         'borderTop': f'3px solid {colors["primary"]}'}),
                    dcc.Tab(label='🕐 Time Patterns', value='time',
                           style={'padding': '12px', 'fontWeight': '600'},
                           selected_style={'padding': '12px', 'fontWeight': '600', 
                                         'borderTop': f'3px solid {colors["primary"]}'}),
                    dcc.Tab(label='📊 Delay Analysis', value='analysis',
                           style={'padding': '12px', 'fontWeight': '600'},
                           selected_style={'padding': '12px', 'fontWeight': '600', 
                                         'borderTop': f'3px solid {colors["primary"]}'})
                ]),
                
                # Tab content
                html.Div(id='tab-content')
            ]
        )
    ])
])

# Callbacks
@app.callback(
    Output('threshold-value', 'children'),
    Input('delay-threshold', 'value')
)
def update_threshold_display(value):
    return f"{value} minutes"

@app.callback(
    [Output('airline-filter', 'value'),
     Output('delay-threshold', 'value'),
     Output('date-range', 'start_date'),
     Output('date-range', 'end_date')],
    [Input('select-all-btn', 'n_clicks'),
     Input('clear-all-btn', 'n_clicks'),
     Input('reset-btn', 'n_clicks')],
    [State('airline-filter', 'value')]
)
def handle_quick_actions(select_all, clear_all, reset, current_airlines):
    from dash import callback_context
    
    if not callback_context.triggered:
        return current_airlines, 15, data['Date'].min(), data['Date'].max()
    
    button_id = callback_context.triggered[0]['prop_id'].split('.')[0]
    
    all_airlines = sorted(data['UniqueCarrier'].unique())
    
    if button_id == 'select-all-btn':
        return all_airlines, 15, data['Date'].min(), data['Date'].max()
    elif button_id == 'clear-all-btn':
        return [], 15, data['Date'].min(), data['Date'].max()
    elif button_id == 'reset-btn':
        return all_airlines, 15, data['Date'].min(), data['Date'].max()
    
    return current_airlines, 15, data['Date'].min(), data['Date'].max()

@app.callback(
    [Output('summary-stats', 'children'),
     Output('tab-content', 'children')],
    [Input('delay-type-dropdown', 'value'),
     Input('airline-filter', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('delay-threshold', 'value'),
     Input('main-tabs', 'value')]
)
def update_dashboard(selected_delay, selected_airlines, start_date, end_date, delay_threshold, active_tab):
    # Filter data
    filtered_data = data[
        (data['UniqueCarrier'].isin(selected_airlines)) &
        (data['Date'] >= start_date) &
        (data['Date'] <= end_date)
    ].copy()
    
    # Check if filtered data is empty
    if len(filtered_data) == 0:
        summary = html.Div(style={
            'textAlign': 'center', 'padding': '40px', 'backgroundColor': '#fff3cd',
            'borderRadius': '12px', 'border': '2px solid #ffc107'
        }, children=[
            html.Div("⚠️", style={'fontSize': '48px', 'marginBottom': '15px'}),
            html.H3("No Data Available", style={'color': '#856404', 'marginBottom': '10px'}),
            html.P("Please adjust your filters to view flight data.", style={'color': '#856404'})
        ])
        
        return summary, html.Div()
    
    # Calculate statistics
    total_flights = len(filtered_data)
    delayed_flights = len(filtered_data[filtered_data[selected_delay] > delay_threshold])
    avg_delay = filtered_data[selected_delay].mean()
    max_delay = filtered_data[selected_delay].max()
    cancelled = filtered_data['Cancelled'].sum()
    delay_pct = (delayed_flights/total_flights*100) if total_flights > 0 else 0
    cancel_pct = (cancelled/total_flights*100) if total_flights > 0 else 0
    
    # Summary cards with enhanced styling
    def create_stat_card(label, value, icon, color, subtitle=""):
        return html.Div(style={
            'backgroundColor': colors['card'],
            'padding': '15px 10px',
            'borderRadius': '10px',
            'boxShadow': '0 2px 8px rgba(0,0,0,0.08)',
            'border': f'1px solid {colors["border"]}',
            'textAlign': 'center',
            'width': '17%',
            'display': 'inline-block',
            'marginRight': '1.25%',
            'transition': 'transform 0.2s',
            'cursor': 'default',
            'position': 'relative',
            'overflow': 'hidden'
        }, children=[
            html.Div(style={
                'position': 'absolute',
                'top': '0',
                'left': '0',
                'width': '100%',
                'height': '3px',
                'backgroundColor': color
            }),
            html.Div(icon, style={'fontSize': '28px', 'marginBottom': '8px'}),
            html.H3(value, style={'color': color, 'margin': '5px 0', 'fontSize': '24px', 'fontWeight': '700'}),
            html.P(label, style={'color': colors['text'], 'fontSize': '12px', 'margin': '0', 'fontWeight': '600'}),
            html.P(subtitle, style={'color': '#95a5a6', 'fontSize': '10px', 'margin': '3px 0 0 0'}) if subtitle else None
        ])
    
    summary = html.Div(style={'marginBottom': '0'}, children=[
        create_stat_card("Total Flights", f"{total_flights:,}", "✈️", colors['primary'], "in selected period"),
        create_stat_card("Delayed Flights", f"{delayed_flights:,}", "⏰", colors['warning'], f"{delay_pct:.1f}% of total"),
        create_stat_card("Average Delay", f"{avg_delay:.1f}m", "📊", colors['info'], "mean delay time"),
        create_stat_card("Maximum Delay", f"{max_delay:.0f}m", "🔴", colors['secondary'], "worst delay"),
        create_stat_card("Cancellations", f"{cancelled:,}", "❌", "#95a5a6", f"{cancel_pct:.3f}% rate")
    ])
    
    # Create charts based on active tab
    if active_tab == 'overview':
        tab_content = create_overview_tab(filtered_data, selected_delay, colors)
    elif active_tab == 'airlines':
        tab_content = create_airlines_tab(filtered_data, selected_delay, colors, delay_threshold)
    elif active_tab == 'time':
        tab_content = create_time_tab(filtered_data, selected_delay, colors, day_names)
    else:  # analysis
        tab_content = create_analysis_tab(filtered_data, selected_delay, colors, delay_threshold)
    
    return summary, tab_content

def create_chart_container(title, figure, description=""):
    return html.Div(style={
        'backgroundColor': colors['card'],
        'padding': '20px',
        'borderRadius': '12px',
        'boxShadow': '0 1px 3px rgba(0,0,0,0.08)',
        'border': f'1px solid {colors["border"]}',
        'marginBottom': '20px'
    }, children=[
        html.H4(title, style={'color': colors['text'], 'margin': '0 0 10px 0', 'fontSize': '18px', 'fontWeight': '600'}),
        html.P(description, style={'color': '#7f8c8d', 'fontSize': '13px', 'marginBottom': '15px'}) if description else None,
        dcc.Graph(figure=figure, config={'displayModeBar': True, 'displaylogo': False})
    ])

def create_overview_tab(filtered_data, selected_delay, colors):
    # Time series
    delay_time = filtered_data.groupby('Date')[selected_delay].agg(['mean', 'count']).reset_index()
    fig_line = go.Figure()
    fig_line.add_trace(go.Scatter(x=delay_time['Date'], y=delay_time['mean'],
                                  mode='lines', name='Daily Average',
                                  line=dict(color=colors['primary'], width=2.5),
                                  fill='tozeroy', fillcolor='rgba(52, 152, 219, 0.1)'))
    fig_line.add_trace(go.Scatter(x=delay_time['Date'], y=delay_time['mean'].rolling(7).mean(),
                                  mode='lines', name='7-Day Trend',
                                  line=dict(color=colors['secondary'], width=3, dash='dash')))
    fig_line.update_layout(title='', xaxis_title='Date', yaxis_title='Average Delay (minutes)',
                          hovermode='x unified', template='plotly_white', height=400,
                          legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    
    # Distribution
    fig_dist = px.histogram(filtered_data[filtered_data[selected_delay] > 0], 
                           x=selected_delay, nbins=50,
                           color_discrete_sequence=[colors['primary']])
    fig_dist.update_layout(title='', xaxis_title='Delay (minutes)', yaxis_title='Number of Flights',
                          template='plotly_white', showlegend=False, height=400)
    
    return html.Div([
        html.Div([
            create_chart_container("Delay Trends Over Time", fig_line, 
                                 "Track how delays change over your selected period with a 7-day moving average trend line")
        ], style={'width': '59%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
        html.Div([
            create_chart_container("Delay Distribution", fig_dist,
                                 "See how delays are distributed across all flights")
        ], style={'width': '39%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ])

def create_airlines_tab(filtered_data, selected_delay, colors, delay_threshold):
    # Airline comparison
    airline_delay = filtered_data.groupby('UniqueCarrier').agg({
        selected_delay: 'mean', 'ID': 'count'
    }).reset_index()
    airline_delay.columns = ['UniqueCarrier', 'AvgDelay', 'FlightCount']
    airline_delay = airline_delay.sort_values('AvgDelay', ascending=True)
    
    fig_bar = go.Figure(data=[
        go.Bar(x=airline_delay['AvgDelay'], y=airline_delay['UniqueCarrier'],
               orientation='h', text=[f"{x:.1f}m" for x in airline_delay['AvgDelay']],
               textposition='auto', marker_color=colors['primary'],
               customdata=airline_delay['FlightCount'],
               hovertemplate='<b>%{y}</b><br>Average Delay: %{x:.1f} min<br>Total Flights: %{customdata:,}<extra></extra>')
    ])
    fig_bar.update_layout(title='', xaxis_title='Average Delay (minutes)', yaxis_title='',
                         template='plotly_white', height=500)
    
    # Top routes
    filtered_data['Route'] = filtered_data['Origin'] + ' → ' + filtered_data['Dest']
    route_delay = filtered_data.groupby('Route').agg({
        selected_delay: 'mean', 'ID': 'count'
    }).reset_index()
    route_delay.columns = ['Route', 'AvgDelay', 'FlightCount']
    route_delay = route_delay[route_delay['FlightCount'] >= 100].nlargest(10, 'AvgDelay')
    
    fig_route = px.bar(route_delay, x='AvgDelay', y='Route', orientation='h',
                      color='AvgDelay', color_continuous_scale='Reds',
                      text=[f"{x:.1f}m" for x in route_delay['AvgDelay']])
    fig_route.update_traces(textposition='auto')
    fig_route.update_layout(title='', xaxis_title='Average Delay (minutes)', yaxis_title='',
                           template='plotly_white', showlegend=False, height=500)
    
    return html.Div([
        html.Div([
            create_chart_container("Airline Performance Comparison", fig_bar,
                                 "Compare average delays across different airlines")
        ], style={'width': '49%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
        html.Div([
            create_chart_container("Top 10 Most Delayed Routes", fig_route,
                                 "Routes with highest average delays (minimum 100 flights)")
        ], style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ])

def create_time_tab(filtered_data, selected_delay, colors, day_names):
    # Heatmap
    heatmap_data = filtered_data.groupby(['DayOfWeek', 'Hour'])[selected_delay].mean().reset_index()
    heatmap_pivot = heatmap_data.pivot(index='DayOfWeek', columns='Hour', values=selected_delay)
    
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=[day_names.get(i, i) for i in heatmap_pivot.index],
        colorscale='RdYlGn_r',
        hovertemplate='<b>%{y}</b><br>Hour: %{x}:00<br>Avg Delay: %{z:.1f} min<extra></extra>',
        colorbar=dict(title="Minutes")
    ))
    fig_heatmap.update_layout(title='', xaxis_title='Hour of Day', yaxis_title='',
                             template='plotly_white', height=400)
    
    # Hourly pattern
    hourly_pattern = filtered_data.groupby('Hour')[selected_delay].mean().reset_index().sort_values('Hour')
    
    fig_hourly = go.Figure(data=[
        go.Scatter(x=hourly_pattern['Hour'], y=hourly_pattern[selected_delay],
                  mode='lines+markers', fill='tozeroy',
                  line=dict(color=colors['primary'], width=3), 
                  marker=dict(size=10, color=colors['primary']),
                  fillcolor='rgba(52, 152, 219, 0.1)',
                  hovertemplate='<b>Hour: %{x}:00</b><br>Avg Delay: %{y:.1f} min<extra></extra>')
    ])
    fig_hourly.update_layout(title='', xaxis_title='Hour of Day (0 = Midnight, 23 = 11 PM)', 
                            yaxis_title='Average Delay (minutes)',
                            xaxis=dict(tickmode='linear', tick0=0, dtick=2, range=[-0.5, 23.5]),
                            template='plotly_white', showlegend=False, height=400)
    
    return html.Div([
        create_chart_container("Delay Patterns: Day of Week vs Hour", fig_heatmap,
                             "Identify when delays are most common throughout the week"),
        html.Div([
            create_chart_container("Hourly Delay Pattern", fig_hourly,
                                 "See how delays vary throughout the day")
        ], style={'width': '100%'})
    ])

def create_analysis_tab(filtered_data, selected_delay, colors, delay_threshold):
    # Delay causes
    delay_causes = filtered_data[filtered_data[selected_delay] > delay_threshold][
        ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    ].sum()
    
    cause_colors = ['#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']
    fig_causes = go.Figure(data=[
        go.Bar(x=delay_causes.index, y=delay_causes.values,
               marker_color=cause_colors,
               text=[f"{x:,.0f}m" for x in delay_causes.values],
               textposition='auto',
               hovertemplate='<b>%{x}</b><br>Total Minutes: %{y:,.0f}<extra></extra>')
    ])
    fig_causes.update_layout(title='', xaxis_title='', yaxis_title='Total Delay Minutes',
                            template='plotly_white', showlegend=False, height=400)
    
    # Cancellation pie
    cancel_data = filtered_data[filtered_data['Cancelled'] == True]['CancellationReason'].value_counts()
    if len(cancel_data) > 0:
        fig_pie = px.pie(values=cancel_data.values, names=cancel_data.index,
                        color_discrete_sequence=px.colors.qualitative.Set3,
                        hole=0.4)
        fig_pie.update_traces(textposition='inside', textinfo='percent+label')
        fig_pie.update_layout(title='', template='plotly_white', height=400)
    else:
        fig_pie = go.Figure()
        fig_pie.add_annotation(text="No Cancellations<br>in Selected Period", 
                              xref="paper", yref="paper",
                              x=0.5, y=0.5, showarrow=False,
                              font=dict(size=18, color='#95a5a6'))
        fig_pie.update_layout(title='', template='plotly_white', height=400)
    
    return html.Div([
        html.Div([
            create_chart_container("Delay Causes Breakdown", fig_causes,
                                 "Total delay minutes attributed to each cause")
        ], style={'width': '59%', 'display': 'inline-block', 'marginRight': '2%', 'verticalAlign': 'top'}),
        html.Div([
            create_chart_container("Cancellation Reasons", fig_pie,
                                 "Distribution of flight cancellations by reason")
        ], style={'width': '39%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ])

# Run the app
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='inline')